In [1]:
!pip install bitsandbytes>=0.45.3

In [2]:
from huggingface_hub import login

# Hugging face token
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

# Log in to Hugging Face Hub
login(token=HF_TOKEN)

In [3]:
from huggingface_hub import snapshot_download

sql_lora_path = snapshot_download(repo_id="google-cloud-partnership/gemma-2-2b-it-lora-sql")

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

In [4]:
import torch
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest

prompts = [
    "The future of AI is",
]
sampling_params = SamplingParams(temperature=0.3, top_p=0.95, max_tokens=256)

model_id = "google/gemma-2-2b"
llm = LLM(model=model_id,dtype=torch.float16,enable_lora=True, trust_remote_code=True,
          kv_cache_dtype="fp8",calculate_kv_scales=True , \
          quantization="bitsandbytes", load_format="bitsandbytes")

outputs = llm.generate(prompts,sampling_params,
                       lora_request=LoRARequest("sql_adapter", 1, sql_lora_path))

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

INFO 05-03 13:07:29 [__init__.py:239] Automatically detected platform cuda.
INFO 05-03 13:07:39 [config.py:2968] Downcasting torch.float32 to torch.float16.
INFO 05-03 13:07:54 [config.py:717] This model supports multiple tasks: {'classify', 'reward', 'embed', 'score', 'generate'}. Defaulting to 'generate'.
WARNING 05-03 13:07:54 [config.py:830] bitsandbytes quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 05-03 13:07:54 [arg_utils.py:1658] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 05-03 13:07:54 [config.py:1403] Using fp8 data type to store kv cache. It reduces the GPU memory footprint and boosts the performance. Meanwhile, it may cause accuracy drop without a proper scaling factor
INFO 05-03 13:07:56 [llm_engine.py:240] Initializing a V0 LLM engine (v0.8.5.post1) with config: model='google/gemma-2-2b', speculative_config=None, tokenizer='google/gemma-2-2b', skip_tokenizer_init=False, tokeni

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 05-03 13:08:47 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 05-03 13:08:47 [model_runner.py:1140] Model loading took 2.1314 GiB and 47.851604 seconds
INFO 05-03 13:09:07 [worker.py:287] Memory profiling takes 19.33 seconds
INFO 05-03 13:09:07 [worker.py:287] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 05-03 13:09:07 [worker.py:287] model weights take 2.13GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 2.36GiB; the rest of the memory reserved for KV Cache is 8.72GiB.
INFO 05-03 13:09:07 [executor_base.py:112] # cuda blocks: 10995, # CPU blocks: 5041
INFO 05-03 13:09:07 [executor_base.py:117] Maximum concurrency for 8192 tokens per request: 21.47x
INFO 05-03 13:09:12 [model_runner.py:1450] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. 

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 05-03 13:11:19 [model_runner.py:1592] Graph capturing finished in 127 secs, took 0.78 GiB
INFO 05-03 13:11:19 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 151.34 seconds


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Prompt: 'The future of AI is', Generated text: ' here. The future of AI is now. The future of AI is here. The future of AI is now. The future of AI is here. The future of AI is now. The future of AI is here. The future of AI is now. The future of AI is here. The future of AI is now. The future of AI is here. The future of AI is now. The future of AI is here. The future of AI is now. The future of AI is here. The future of AI is now. The future of AI is here. The future of AI is now. The future of AI is here. The future of AI is now. The future of AI is here. The future of AI is now. The future of AI is here. The future of AI is now. The future of AI is here. The future of AI is now. The future of AI is here. The future of AI is now. The future of AI is here. The future of AI is now. The future of AI is here. The future of AI is now. The future of AI is here. The future of AI is now. The future of AI is here. The future of AI is now. The future of AI is here. The future'


In [5]:
from huggingface_hub import HfApi, login
import os

login(token=HF_TOKEN)  # Replace with your actual token

# Define the repository name on Hugging Face
repo_name = "Tharun013/gemma2-2b-finetuned-sql"

# Ensure the LoRA adapter directory exists
if not os.path.exists(sql_lora_path):
    raise ValueError(f"LoRA adapter path {sql_lora_path} does not exist. Please save the adapter first.")

# Initialize the HfApi client
api = HfApi()

# Create the repository if it doesn't exist (optional)
api.create_repo(repo_id=repo_name, exist_ok=True)

# Upload the LoRA adapter directory to the Hugging Face Hub
api.upload_folder(
    folder_path=sql_lora_path,
    repo_id=repo_name,
    commit_message="Pushed gemma2-2b finetuned LoRA adapter - v0",
    repo_type="model"
)

print(f"Successfully pushed LoRA adapter to {repo_name}")

No files have been modified since last commit. Skipping to prevent empty commit.


Successfully pushed LoRA adapter to Tharun013/gemma2-2b-finetuned-sql
